In [1]:
from meta_planning.parsers import parse_trajectory, parse_model, parse_plan
from meta_planning import ValidationTask
from meta_planning import dataset
from meta_planning.pddl import SensorModel, Plan
from meta_planning.functions import generate_all_literals, get_matching_literals

import numpy as np
import glob

In [2]:
domain = "blocks2"

M = parse_model("benchmarks/%s/domain.pddl" % domain)

In [3]:
trajectories = sorted(glob.glob("benchmarks/%s/trajectories/*" % domain))

for i in range(len(trajectories)):

    t = parse_trajectory(trajectories[i], M)
    T = [t]

    
    # Base Observability
    F = generate_all_literals(M.predicates, T[0].objects, M.types)

    mapping = {f:f for f in F}
    mapping.update({f.negate():f.negate() for f in F})

    S = SensorModel(mapping,[0,0,1])
    
    
    
    preds = [p for p in M.predicates if p.name in ["on", "ontable", "clear"]]

    objects = [o for o in T[0].objects if o.type_name == "block"]

    subF = get_matching_literals(F, preds, objects)

    mapping = {f:f for f in subF}

    S.set_observability(mapping, [1,0,0])
    
    
    # Low Observability
    preds = [p for p in M.predicates if p.name in ["holding"]]

    objects = [o for o in T[0].objects if o.type_name == "nrobot"]

    subF = get_matching_literals(F, preds, objects)

    mapping = {f:f for f in subF}

    S.set_observability(mapping, [0.4,0,0.6])
    
    
    # High Observability
    preds = [p for p in M.predicates if p.name in ["holding"]]

    objects = [o for o in T[0].objects if o.type_name == "orobot"]

    subF = get_matching_literals(F, preds, objects)

    mapping = {f:f for f in subF}

    S.set_observability(mapping, [0.6,0,0.4])
    
    
#     for k,v in S.observability_table.items():
#         print(k,v)
    
    
    # Generate observation
    O = [t.observe_with_sensor_model(S, action_observability=0) for t in T]
    
    
    # Solve with sensor model
    valtask = ValidationTask(M,O, compile_known=False, sensor_model=S, use_cost=True)
    sol = valtask.validate(clean=True, planner="metric-ff", t=300)
    if sol.solution_found:
        plan_SM = sol.solution_plan.clean("sense").clean("validate")
    else:
        plan_SM = Plan([])
    
    plan_SM.to_file("benchmarks/%s/plans/plan-%s-SM" % (domain, str(i).zfill(3)))
    
    
    # Solve without sensor model
    valtask = ValidationTask(M,O, compile_known=False)
    sol2 = valtask.validate(clean=True, parallel=False, planner="madagascar", t=300)
    if sol2.solution_found:
        plan_NO = sol2.solution_plan.clean("validate")
    else:
        plan_NO = Plan([])
    
    plan_NO.to_file("benchmarks/%s/plans/plan-%s-NO" % (domain, str(i).zfill(3)))
    
    
    # Original plan
    original_actions = [s.next_action for s in t.states if s.next_action is not None]
    plan_OR = Plan(original_actions)
    
    plan_OR.to_file("benchmarks/%s/plans/plan-%s-OR" % (domain, str(i).zfill(3)))
    
    

ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 21 -P 0 > planner_out
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 21 -P 0 > planner_out
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F

ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 21 -P 0 > planner_out
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F 21 -P 0 > planner_out
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 300; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/madagascar/M compiled_domain compiled_problem -S 1 -Q -o solution_plan -F